In [3]:
import os 
import logging
import scrapy
import requests
from scrapy.crawler import CrawlerProcess
import json
import pandas as pd
import plotly.express as px
import glob
import boto3

# Cities and creation of df

In [4]:
city_names= ["Mont Saint Michel",
"St Malo",
"Bayeux",
"Le Havre",
"Rouen",
"Paris",
"Amiens",
"Lille",
"Strasbourg",
"Chateau du Haut Koenigsbourg",
"Colmar",
"Eguisheim",
"Besancon",
"Dijon",
"Annecy",
"Grenoble",
"Lyon",
"Gorges du Verdon",
"Bormes les Mimosas"
]

In [5]:
df = pd.DataFrame(columns=['city_id' , 'name', 'latitude', 'longitude', 'main_weather', 'expected_rain', 'day_temperature'])
df

,city_id,name,latitude,longitude,main_weather,expected_rain,day_temperature


# Coordinates, weather info, convert df to csv

In [6]:
api_key = "f482d2b26c2e58bf480acbe962e74f18"

In [7]:
for i in range(len(city_names)):
    print("making request: {}".format(city_names[i]))
    
#get latitude and longitude of cities    
    
    par = {
        "city": city_names[i],
        "country": "France",
        "format": "json"
          }
    
    r = requests.get('https://nominatim.openstreetmap.org/search', params=par)
    res = r.json()
    
#fix lat and lon
    df.loc[i,'city_id'] = i
    df.loc[i,'name'] = city_names[i]
    df.loc[i, 'latitude'] = res[0]['lat']
    df.loc[i, 'longitude'] = res[0]['lon']
    
#get weather
    par = {
        "lat": df.loc[i, 'latitude'],
        "lon": df.loc[i, 'longitude'],
        "exclude": "current,minutely,hourly",
        "units": "metric",
        "appid": api_key,
           }
    
    r = requests.get('https://api.openweathermap.org/data/2.5/onecall', params=par)
    res = r.json()
    
    # Compute expected volume of rain
    expected_rain = 0
    print(res) 
    
    for d in res['daily']:
        if 'rain' in d.keys():
            expected_rain += d['pop']*d['rain']
            
    # Compute average day temperature
    temperatures = pd.Series([d['temp']['day'] for d in res['daily']])
    mean_temperature = temperatures.mean()

    # Extract most probable weather
    weathers = pd.Series([d['weather'][0]['main'] for d in res['daily']])
    main_weather = weathers.mode()[0]
    
    df.loc[i, 'main_weather'] = main_weather
    df.loc[i,'expected_rain'] = expected_rain
    df.loc[i,'day_temperature'] = mean_temperature


making request: Mont Saint Michel
{'lat': 48.636, 'lon': -1.5115, 'timezone': 'Europe/Paris', 'timezone_offset': 7200, 'daily': [{'dt': 1655899200, 'sunrise': 1655870619, 'sunset': 1655928738, 'moonrise': 1655858820, 'moonset': 1655904900, 'moon_phase': 0.79, 'temp': {'day': 22.6, 'min': 14.9, 'max': 24.13, 'night': 17, 'eve': 22.21, 'morn': 18.18}, 'feels_like': {'day': 22.74, 'night': 17.18, 'eve': 22.47, 'morn': 18.35}, 'pressure': 1011, 'humidity': 70, 'dew_point': 16.85, 'wind_speed': 3.96, 'wind_deg': 119, 'wind_gust': 6.27, 'weather': [{'id': 501, 'main': 'Rain', 'description': 'moderate rain', 'icon': '10d'}], 'clouds': 81, 'pop': 0.91, 'rain': 11.22, 'uvi': 6.76}, {'dt': 1655985600, 'sunrise': 1655957034, 'sunset': 1656015147, 'moonrise': 1655946180, 'moonset': 1655995500, 'moon_phase': 0.82, 'temp': {'day': 23.34, 'min': 14.66, 'max': 24.22, 'night': 15.69, 'eve': 20.29, 'morn': 16.32}, 'feels_like': {'day': 23.17, 'night': 15.66, 'eve': 20.25, 'morn': 16.49}, 'pressure': 101

IndexError: list index out of range

In [ ]:
df

In [ ]:
df.loc[:,'rank'] = df['expected_rain'].rank(method='min')
df.loc[:,'inverted_rank'] = df['expected_rain'].rank(method='min', ascending=False)
df = df.sort_values(by=['expected_rain', 'day_temperature'], ascending = [True, False]).reset_index(drop=True)
display(df)

,city_id,name,latitude,longitude,main_weather,expected_rain,day_temperature,rank,inverted_rank
0,1,St Malo,48.649518,-2.0260409,Rain,16.4615,19.0925,1.0,17.0
1,2,Bayeux,49.2764624,-0.7024738,Rain,16.8455,19.45125,2.0,16.0
2,5,Paris,48.8588897,2.3200410217200766,Rain,17.8608,22.72125,3.0,15.0
3,3,Le Havre,49.4938975,0.1079732,Rain,18.65,18.45625,4.0,14.0
4,0,Mont Saint Michel,48.6359541,-1.511459954959514,Rain,19.8474,19.975,5.0,13.0
5,4,Rouen,49.4404591,1.0939658,Rain,21.7818,21.57875,6.0,12.0
6,6,Amiens,49.8941708,2.2956951,Rain,23.0233,21.06625,7.0,11.0
7,7,Lille,50.6365654,3.0635282,Rain,34.6325,20.8025,8.0,10.0
8,15,Grenoble,45.1875602,5.7357819,Rain,36.9146,26.46125,9.0,9.0
9,16,Lyon,45.7578137,4.8320114,Rain,42.1342,23.44125,10.0,8.0


In [ ]:
print('Best places for a trip next week are : ')
print()

for i,row in df.loc[df['rank']==1,:].iterrows():
    print("{} -- Mostly {} with temperature {} °C".format(row['name'], row['main_weather'], row['day_temperature']))

Best places for a trip next week are : 

St Malo -- Mostly Rain with temperature 19.092499999999998 °C


In [ ]:
df = df.fillna(0)

In [ ]:
df.loc[:,'latitude'] = df['latitude'].astype('float')
df.loc[:,'longitude'] = df['longitude'].astype('float')
df.loc[:,'expected_rain'] = df['expected_rain'].astype('float')
df.loc[:,'day_temperature'] = df['day_temperature'].astype('float')
df.loc[:,'rank'] = df['rank'].astype('int')
df.loc[:,'inverted_rank'] = df['inverted_rank'].astype('int')

In [8]:
fig = px.scatter_mapbox(df, lat="latitude", lon="longitude", hover_name = 'name', zoom = 4,
                        hover_data = ['main_weather', 'expected_rain', 'day_temperature'], 
                        color = 'day_temperature', color_continuous_scale = 'Bluered', size = 'inverted_rank',
                        mapbox_style="carto-positron")
fig.show()

ValueError: Value of 'size' is not the name of a column in 'data_frame'. Expected one of ['city_id', 'name', 'latitude', 'longitude', 'main_weather', 'expected_rain', 'day_temperature'] but received: inverted_rank

In [12]:
df.to_csv('destinations.csv', index=False)

# Scrapping Booking.com for hotel information

### La Rochelle

In [9]:
city = 'La Rochelle'

In [10]:
r = requests.get('https://www.booking.com/index.fr.html')
r

<Response [200]>

In [15]:
class BookingSpider(scrapy.Spider):
    name = "booking"

    start_urls = ['https://www.booking.com/index.fr.html']

    def parse(self, response):
        return scrapy.FormRequest.from_response(
            response,
            formdata={'ss': city},
            callback=self.after_search
        )

    def after_search(self, response):
        
        booking = response.css('.sr_item')

        for data in booking:
            yield {
                'name': data.css('.sr-hotel__name::text').get(),
                'url': 'https://www.booking.com' + data.css('.hotel_name_link').attrib["href"],
                'coords': data.css('.sr_card_address_line a').attrib["data-coords"],
                'score': data.css('.bui-review-score__badge::text').get(),
                'description': data.css('.hotel_desc::text').get()
                
            }
        
        
        try:
            next_page = response.css('a.paging-next').attrib["href"]
        except KeyError:
            logging.info('No next page. Terminating crawling process.')
        else:
            yield response.follow(next_page, callback=self.after_search)

In [16]:
filename = "hotels_" + city.replace(" ", "-") + ".json"


process = CrawlerProcess(settings = {
    'USER_AGENT': 'Chrome/84.0 (compatible; MSIE 7.0; Windows NT 5.1)',
    'LOG_LEVEL': logging.INFO,
    "FEEDS": {
        'res/' + filename: {"format": "json"},
    }
})

process.crawl(BookingSpider)
process.start()

2022-06-22 12:25:54 [scrapy.utils.log] INFO: Scrapy 2.6.1 started (bot: scrapybot)
2022-06-22 12:25:54 [scrapy.utils.log] INFO: Versions: lxml 4.8.0.0, libxml2 2.9.12, cssselect 1.1.0, parsel 1.6.0, w3lib 1.21.0, Twisted 22.2.0, Python 3.9.12 (main, Apr  5 2022, 01:53:17) - [Clang 12.0.0 ], pyOpenSSL 21.0.0 (OpenSSL 1.1.1o  3 May 2022), cryptography 3.4.8, Platform macOS-10.16-x86_64-i386-64bit
2022-06-22 12:25:54 [scrapy.crawler] INFO: Overridden settings:
{'LOG_LEVEL': 20,
 'USER_AGENT': 'Chrome/84.0 (compatible; MSIE 7.0; Windows NT 5.1)'}
2022-06-22 12:25:55 [scrapy.extensions.telnet] INFO: Telnet Password: 28d207a7c3a1e4e2
2022-06-22 12:25:55 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats']
2022-06-22 12:25:55 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scra

### Paris

In [17]:
city = 'Paris'

In [18]:
r = requests.get('https://www.booking.com/index.fr.html')
r

<Response [200]>

In [11]:
class BookingSpider(scrapy.Spider):
    name = "booking"

    start_urls = ['https://www.booking.com/index.fr.html']

    def parse(self, response):
        return scrapy.FormRequest.from_response(
            response,
            formdata={'ss': city},
            callback=self.after_search
        )

    def after_search(self, response):
        
        booking = response.css('.sr_item')

        for b in booking:
            yield {
                'name': b.css('.sr-hotel__name::text').get(),
                'url': 'https://www.booking.com' + b.css('.hotel_name_link').attrib["href"],
                'coords': b.css('.sr_card_address_line a').attrib["data-coords"],
                'score': b.css('.bui-review-score__badge::text').get(),
                'description': b.css('.hotel_desc::text').get()
                
            }
        
        
        try:
            next_page = response.css('a.paging-next').attrib["href"]
        except KeyError:
            logging.info('No next page. Terminating crawling process.')
        else:
            yield response.follow(next_page, callback=self.after_search)

In [12]:
filename = "hotels_" + city.replace(" ", "-") + ".json"


process = CrawlerProcess(settings = {
    'USER_AGENT': 'Chrome/84.0 (compatible; MSIE 7.0; Windows NT 5.1)',
    'LOG_LEVEL': logging.INFO,
    "FEEDS": {
        'res/' + filename: {"format": "json"},
    }
})

process.crawl(BookingSpider)
process.start()

2022-06-22 12:28:43 [scrapy.utils.log] INFO: Scrapy 2.6.1 started (bot: scrapybot)
2022-06-22 12:28:43 [scrapy.utils.log] INFO: Versions: lxml 4.8.0.0, libxml2 2.9.12, cssselect 1.1.0, parsel 1.6.0, w3lib 1.21.0, Twisted 22.2.0, Python 3.9.12 (main, Apr  5 2022, 01:53:17) - [Clang 12.0.0 ], pyOpenSSL 21.0.0 (OpenSSL 1.1.1o  3 May 2022), cryptography 3.4.8, Platform macOS-10.16-x86_64-i386-64bit
2022-06-22 12:28:43 [scrapy.crawler] INFO: Overridden settings:
{'LOG_LEVEL': 20,
 'USER_AGENT': 'Chrome/84.0 (compatible; MSIE 7.0; Windows NT 5.1)'}
2022-06-22 12:28:43 [scrapy.extensions.telnet] INFO: Telnet Password: a4905e95063e016a
2022-06-22 12:28:43 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats']
2022-06-22 12:28:44 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scra

# Data lake with S3

In [ ]:
cities = pd.read_csv('destinations.csv')
cities.head()

In [ ]:
hotel_files = glob.glob('res/_hotels_*.json')

hotels = pd.DataFrame(columns = ['city_id', 'city_name', 'hotel_id', 'name', 'url', 'latitude', 'longitude', 'score', 'description'])
hotels


In [ ]:
for f in hotel_files:
    city_name = f.split('_')[2].split('.')[0].replace("-"," ")

In [ ]:
for f in hotel_files:
    city_id = cities.loc[cities['name'] == city_name,'city_id'].values[0]

In [ ]:
hotel_files

In [ ]:
f1 = pd.read_json('res/hotels_Paris.json')
hotels=pd.DataFrame(f1)
hotels['name']=(hotels['name']).astype(str).replace('\n','',regex=True)
hotels['description']=(hotels['description']).astype(str).replace('\n','',regex=True)
hotels.head()

In [21]:
split_data = hotels['coords'].str.split(',')
hotels.loc[:,'latitude'] = split_data.apply(lambda x: x[1])
hotels.loc[:,'longitude'] = split_data.apply(lambda x: x[0])
split_data




NameError: name 'hotels' is not defined

In [ ]:
#hotels_all = hotels_all.append(hotels)

In [ ]:
del hotels['coords']
hotels.head()

In [ ]:
hotels.loc[:, 'city_name']='Paris'
hotels.loc[:, 'city_id'] = cities.loc[cities['name'] == city_name,'city_id'].values[0]
hotels.loc[:, 'latitude'] = hotels['latitude'].astype('float')
hotels.loc[:, 'longitude'] = hotels['longitude'].astype('float')
hotels.loc[:, 'score'] = hotels['score'].str.replace(',','.').astype('float')

In [ ]:
hotels.head()

In [ ]:
hotels.to_csv('res/0_hotels.csv', index=False)

In [ ]:

hotels_with_score = hotels.loc[hotels['score']>9.5,:]

fig = px.scatter_mapbox(hotels_with_score, lat="latitude", lon="longitude", hover_name = 'name', zoom = 4,
                        hover_data = ['description'],
                        color = 'score', color_continuous_scale = 'thermal',
                        mapbox_style="carto-positron")
fig.show()